In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Perceptron
from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('jugadores_ready.csv')

In [7]:
df

,Titulares,T,TaP,C,CP,CF,Reg,PA,partido,equipo,Jornada
0,0,0.0,0.0,0.0,0.0,0.0,0,0,1,girona,1
1,1,0.0,0.0,0.0,0.0,0.0,0,0,1,girona,1
2,2,0.0,0.0,0.0,0.0,0.0,0,1,1,girona,1
3,3,1.0,0.0,2.0,0.0,0.0,1,2,1,girona,1
4,4,0.0,0.0,6.0,1.0,1.0,2,0,1,girona,1
...,...,...,...,...,...,...,...,...,...,...,...
8355,94,0.0,0.0,1.0,0.0,0.0,0,1,380,barcelona,38
8356,2500,1.0,0.0,0.0,0.0,0.0,0,0,380,barcelona,38
8357,2501,0.0,0.0,1.0,0.0,0.0,1,0,380,barcelona,38
8358,1193,0.0,0.0,0.0,0.0,0.0,0,2,380,barcelona,38


In [4]:
list_equip = list(df['equipo'].unique())

In [36]:
def Team(E):
    partidos = df.groupby('equipo').get_group(E)
    x = 1 #jornada a empezar
    lista = []
    for i in range(36): #numero de filas
        datos_a = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.Jornada == i+x].iloc[0:12,0:7]))).T
        datos_a = datos_a.reset_index()
        corners_a = partidos[partidos.Jornada == i+x].iloc[0:12,-1].sum()#suma de los corners
        datos_a.insert(6,'corners', corners_a)


        datos_b = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.Jornada == i+x+1].iloc[0:12,0:7]))).T
        datos_b = datos_b.reset_index()
        corners_b = partidos[partidos.Jornada == i+x+1].iloc[0:12,-1].sum()#suma de los corners
        datos_b.insert(6,'corners', corners_b)

        jugadores_c = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.Jornada == i+x+2].iloc[0:12,0:1]))).T
        corners_c = partidos[partidos.Jornada == i+x+2].iloc[0:12,-1].sum()
        jugadores_c.insert(11,'corners', corners_c)

        total_v1 = pd.concat([datos_a,datos_b,jugadores_c],axis=1,ignore_index=True)
        total_v1 = total_v1.drop([79,86,87,88,89,91],axis=1)
        lista.append(total_v1)

    df_final = pd.concat(lista)
    df_final = df_final.reset_index()

    df_final = df_final.drop(0,axis=1)
    df_final = df_final.drop('index',axis=1)
    df_final = df_final.replace(np.nan,0)
    return df_final

In [37]:
def Perceptron(X):
    #df_final=X    
    df_final = Team(X)
    
    #replace el numero de corners por 3 grupos
    df_final[169] = df_final[169].replace((0,1,2,3),0)
    df_final[169] = df_final[169].replace((4,5,6),1)
    df_final[169] = df_final[169].replace((7,8,9,10,11,12,13),2)
    
    #Selecciona la X y la y
    X = df_final.iloc[:,:162]
    y = df_final.iloc[:,162:163]
    X = np.array(X)
    y = np.array(y)
    y = y[:, 0]
    
    
    #Leave One Out
    loo = LeaveOneOut()
    loo.get_n_splits(X)
    yhat = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]


        #CLASIFICADOR Perceptron (uso el SGD con el parametro perceptron porque el otro no va)
        clf = linear_model.SGDClassifier(loss="perceptron", eta0=1, learning_rate="constant", penalty=None)
        clf.fit(X_train, y_train)
        yhat1 = clf.predict(X_test)
        yhat.append(yhat1)
        
        
        

    return accuracy_score(y, yhat)

In [38]:
Perceptron('barcelona')

0.19444444444444445

In [39]:
#Lista de accuracy de todos los equipos
accuracy_list = []
for i in list_equip:
    accuracy_list.append(Perceptron(i))
accuracy_list

[0.5833333333333334,
 0.3333333333333333,
 0.3055555555555556,
 0.4444444444444444,
 0.19444444444444445,
 0.5555555555555556,
 0.5,
 0.5833333333333334,
 0.19444444444444445,
 0.4444444444444444,
 0.3055555555555556,
 0.4722222222222222,
 0.5555555555555556,
 0.3055555555555556,
 0.2777777777777778,
 0.19444444444444445,
 0.3055555555555556,
 0.2222222222222222,
 0.6388888888888888,
 0.3333333333333333]

In [40]:
Dicc = dict(zip(list_equip,accuracy_list))
Dicc

{'girona': 0.5833333333333334,
 'valladolid': 0.3333333333333333,
 'betis': 0.3055555555555556,
 'levante': 0.4444444444444444,
 'celta': 0.19444444444444445,
 'espanyol': 0.5555555555555556,
 'villarreal': 0.5,
 'real_sociedad': 0.5833333333333334,
 'barcelona': 0.19444444444444445,
 'alaves': 0.4444444444444444,
 'eibar': 0.3055555555555556,
 'huesca': 0.4722222222222222,
 'rayo': 0.5555555555555556,
 'sevilla': 0.3055555555555556,
 'real_madrid': 0.2777777777777778,
 'getafe': 0.19444444444444445,
 'valencia': 0.3055555555555556,
 'atletico': 0.2222222222222222,
 'athletic': 0.6388888888888888,
 'leganes': 0.3333333333333333}

In [41]:
#media del accuracy
sum(accuracy_list)/20

0.38749999999999996